SCENARIUSZ

Firma PwC otrzymała od jednego z klientów zadanie wsparcia jej programu akceleracyjnego. Pierwszym
etapem projektu ma być napisanie modułu dokonującego negatywnej selekcji kandydatów. Na podstawie
danych z roku 2014 spreparowaliśmy zbior danych (dane zostały wybrane według ustalonych kryteriów)
firm wraz z informacją czy firmie udało się przetrwać do 1.01.2016. Twoim zadaniem jest zbudowanie
modelu predykcyjnego na podstawie tych danych. Masz określić prawdopodobieństwo, że dana firma
upadnie w ciągu dwóch lat.

Efekty swojej pracy podsumuj w formie zwięzłej prezentacji w języku angielskim (ok. 5 slajdów), której
adresatem będzie menedżer odpowiedzialny za relacje z klientem. Prezentacja powinna obejmować
kluczowe elementy Twojej pracy, m.in. opis analizy danych, metodologii, wyników modelu, rekomendacji,
itp. Prezentację prześlij jako załącznik w formacie PDF.

ZMIENNE

- GEO - dane geograficzne (pochodzą z poprzedniego projektu, wykonywanego przez jedną z innych firm wielkiej czwórki),
- SECTOR - typ działalności,
- EMPLOYEES - liczba pracowników,
- INCOME - przychód w 2014,
- MARKETING_SPENDING - pieniądze wydane na marketing,
- ACC_BEFORE - czy korzystała wcześniej z wsparcia akceleracyjnego,
- PWC_PRESS_INDEX - wartość wyznaczonego przez dział PWC PRESS CONTACT indeksu pozytywnego przekazu w mediach,
- FB_LIKES - ilość polubień na facebooku,
- TWT_FOLLOWERS - followersi na Twitterze,
- YEAR_FOUNDED - rok założenia,
- EXP_CEO - lata doświadczenia prezesa na podstawie serwisu LinkedIn,
- AREA - powierzchnia biura,
- PWC_EMPLOYEES - ilość pracowników PwC wywodzących się z firmy,
- CREDIT - wartość niespłaconych kredytów,
- FLAG - informacja czy firma upadła do 1.01.2016 r.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('./acceleration_data.csv')
# data

Input data has issiue with quoting. Each column apart from "ID" is quoted twice, whitch makes reading geospacial data difficult. With default options pandas detected only one column. 

To fix this issiue data was exported to .csv file without adding quotes and then read again using first column as index.

In [2]:
data.to_csv('data.csv', index=False, sep='|', quoting=3, escapechar='|')
data = pd.read_csv("./data.csv", index_col=0)
# data

No columns seems to be irrelevant. 

Checked how many "NaN" values dataframe contains. On first glance only EXP_CEO column seemed to contain any.

In [3]:
data[data.isnull().any(axis=1)]

,GEO,SECTOR,EMPLOYEES,INCOME,MARKETING_SPENDING,ACC_BEFORE,PWC_PRESS_INDEX,FB_LIKES,TWT_FOLLOWERS,YEAR_FOUNDED,EXP_CEO,AREA,PWC_EMPLOYEES,CREDIT,FLAG
ID,,,,,,,,,,,,,,,
1,"21.0017163707095 , 52.2314057981037",PUB,2,150426.0,7964,0,-8.078335,1633,138,2010,NaN,32,0,19950,0
2,"21.0068688148316 , 52.2332658119407",PUB,6,500078.0,30212,0,17.032851,6128,605,2011,NaN,44,1,45104,1
3,"21.0746074383869 , 52.2790054865787",IT,27,199767.0,4970,0,-7.321621,487,106,2005,NaN,83,0,499,0
4,"21.0120824468674 , 52.2320822895085",IT,5,109200.0,7010,1,1.135431,661,116,2011,NaN,21,1,0,1
11,"21.1978283467684 , 52.2189889051416",IT,4,80017.0,7598,0,1.213712,809,171,2010,NaN,82,0,19691,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,"21.0993971572514 , 52.2744166403217",RESTAURANT,4,190900.0,7910,0,-9.679458,1632,165,2000,NaN,29,2,0,1
9993,"21.0254303761614 , 52.2280231507402",PUB,2,150506.0,8076,0,7.943083,1670,167,2007,NaN,32,0,0,0
9994,"21.1581620193655 , 52.2035076615214",CAFE,4,149732.0,8057,0,12.182820,1569,129,2010,NaN,26,0,14900,0


Filled missing values with forward-fill method.

In [4]:
df = data.copy(deep=True)
df.EXP_CEO = df.EXP_CEO.fillna(method='bfill')
# df[df.isnull().any(axis=1)]
df.isnull().sum()

GEO                   0
SECTOR                0
EMPLOYEES             0
INCOME                0
MARKETING_SPENDING    0
ACC_BEFORE            0
PWC_PRESS_INDEX       0
FB_LIKES              0
TWT_FOLLOWERS         0
YEAR_FOUNDED          0
EXP_CEO               0
AREA                  0
PWC_EMPLOYEES         0
CREDIT                0
FLAG                  0
dtype: int64

Selected numerical columns to detect outliers.

In [5]:
numerical_columns = [col for col in df.columns if (df[col].dtype=='int64' or df[col].dtype=='float64') and col != 'FLAG']
numerical_columns

['EMPLOYEES',
 'INCOME',
 'MARKETING_SPENDING',
 'ACC_BEFORE',
 'PWC_PRESS_INDEX',
 'FB_LIKES',
 'TWT_FOLLOWERS',
 'YEAR_FOUNDED',
 'EXP_CEO',
 'AREA',
 'PWC_EMPLOYEES',
 'CREDIT']

In [6]:
df[numerical_columns].describe().loc[['min','max', 'mean','50%'],:]

,EMPLOYEES,INCOME,MARKETING_SPENDING,ACC_BEFORE,PWC_PRESS_INDEX,FB_LIKES,TWT_FOLLOWERS,YEAR_FOUNDED,EXP_CEO,AREA,PWC_EMPLOYEES,CREDIT
min,1.0000,-900.0000,830.0000,0.0000,-38.793253,91.0000,16.0000,1989.0000,1.0000,11.0000,0.0000,-9.830000e+03
max,63.0000,647000.0000,68180.0000,1.0000,59.938575,6893.0000,1410.0000,2014.0000,21.0000,387.0000,15.0000,1.313101e+07
mean,12.9563,232443.3755,13515.3352,0.3364,-0.681005,1919.4968,270.4765,2006.9072,6.6498,91.0146,1.5025,2.775444e+05
50%,5.0000,199734.0000,8038.0000,0.0000,-1.186027,1223.0000,165.0000,2007.0000,6.0000,63.0000,1.0000,1.207000e+03


Negative debt was accepted on assumption that values presented in CREDIT column mean net debt.

Since there is only one company with negative income it was deleted.

In [7]:
df[df.INCOME == df.INCOME.min()]

,GEO,SECTOR,EMPLOYEES,INCOME,MARKETING_SPENDING,ACC_BEFORE,PWC_PRESS_INDEX,FB_LIKES,TWT_FOLLOWERS,YEAR_FOUNDED,EXP_CEO,AREA,PWC_EMPLOYEES,CREDIT,FLAG
ID,,,,,,,,,,,,,,,
7239,"21.0040367239336 , 52.2360502686957",IT,5,-900.0,11030,1,2.435256,1113,230,2012,4.0,25,2,0,0


In [8]:
df = df.drop(df[df.INCOME == df.INCOME.min()].index)
df[df.INCOME == df.INCOME.min()]

,GEO,SECTOR,EMPLOYEES,INCOME,MARKETING_SPENDING,ACC_BEFORE,PWC_PRESS_INDEX,FB_LIKES,TWT_FOLLOWERS,YEAR_FOUNDED,EXP_CEO,AREA,PWC_EMPLOYEES,CREDIT,FLAG
ID,,,,,,,,,,,,,,,
8565,"21.0140920062982 , 52.2275265593873",IT,5,11200.0,6020,0,0.857218,576,119,2012,4.0,19,2,-2997,0


Coordinates 

In [9]:
# df.GEO.to_csv('geo.csv')
df.GEO.to_csv('geo.csv', index=False, sep='|', quoting=3, escapechar='|')


In [10]:
geo = pd.read_csv('./geo.csv')
geo.to_csv('geo.csv')

In [11]:

# plt.scatter(geo.x, geo.y)
# plt.show()

In [12]:
# df.drop(['GEO'], axis=1)
df

,GEO,SECTOR,EMPLOYEES,INCOME,MARKETING_SPENDING,ACC_BEFORE,PWC_PRESS_INDEX,FB_LIKES,TWT_FOLLOWERS,YEAR_FOUNDED,EXP_CEO,AREA,PWC_EMPLOYEES,CREDIT,FLAG
ID,,,,,,,,,,,,,,,
1,"21.0017163707095 , 52.2314057981037",PUB,2,150426.0,7964,0,-8.078335,1633,138,2010,5.0,32,0,19950,0
2,"21.0068688148316 , 52.2332658119407",PUB,6,500078.0,30212,0,17.032851,6128,605,2011,5.0,44,1,45104,1
3,"21.0746074383869 , 52.2790054865787",IT,27,199767.0,4970,0,-7.321621,487,106,2005,5.0,83,0,499,0
4,"21.0120824468674 , 52.2320822895085",IT,5,109200.0,7010,1,1.135431,661,116,2011,5.0,21,1,0,1
5,"21.0001250609641 , 52.2399573325692",IT,3,112200.0,9000,0,0.950873,926,172,2012,5.0,23,0,-997,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,"21.0245139057341 , 52.2359749760199",IT,8,79900.0,8980,0,0.974774,888,180,2012,5.0,18,2,2004,1
9997,"21.0186472133435 , 52.2313234646758",IT,19,336500.0,23940,0,4.389039,2383,496,2006,5.0,202,5,0,1
9998,"21.0660097305197 , 52.2955965697952",IT,3,200265.0,4981,1,-4.523559,502,109,2002,5.0,74,1,499,0
